In [1]:
import pandas as pd
import openai
import json
import time
import matplotlib.pyplot as plt
from tqdm import tqdm  # Import tqdm for the progress bar
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score
from sklearn.metrics import balanced_accuracy_score
import numpy as np

In [2]:
#pip install openai

In [3]:
from openai import OpenAI

In [4]:
openai.__version__


'1.1.1'

### Read in annotated data

In [5]:
df = pd.read_csv("../data/prodigy/annotated_output/final/full_combined_dataset_1996.csv", index_col=0)

In [6]:
df.head()

,pmid,journal_name,title,abstract,accepted_label
0,18379746,Der Radiologe,[Pediatric stroke].,Stroke in childhood has gained increasingly mo...,Non-systematic-review
1,24660674,Journal of consulting and clinical psychology,Treatment engagement and response to CBT among...,"In the current study, we compared measures of ...",Human-RCT-non-drug-intervention
2,20159133,Archives of physical medicine and rehabilitation,Relationship between perceived exertion and ph...,To investigate the strength of the relationshi...,Remaining
3,11781147,Biochimica et biophysica acta,Characterization of a missense mutation at his...,Genetic defects in pyruvate dehydrogenase comp...,Remaining
4,31706919,Epilepsy & behavior : E&B,The role of P-glycoprotein (P-gp) and inwardly...,Sudden unexpected death in epilepsy (SUDEP) is...,Non-systematic-review


In [7]:
# Combine the columns
# Implementing custom tags for the combination of journal name, title, and abstract
df['input_journal_title_abstract'] = '<journal>' + df['journal_name'] + '</journal>' + \
                                     '<title>' + df['title'] + '</title>' + \
                                     '<abstract>' + df['abstract'] + '</abstract>'

# Implementing custom tags for the combination of title and abstract only
df['input_title_abstract'] = '<title>' + df['title'] + '</title>' + \
                             '<abstract>' + df['abstract'] + '</abstract>'

In [8]:
df.head(5)

,pmid,journal_name,title,abstract,accepted_label,input_journal_title_abstract,input_title_abstract
0,18379746,Der Radiologe,[Pediatric stroke].,Stroke in childhood has gained increasingly mo...,Non-systematic-review,<journal>Der Radiologe</journal><title>[Pediat...,<title>[Pediatric stroke].</title><abstract>St...
1,24660674,Journal of consulting and clinical psychology,Treatment engagement and response to CBT among...,"In the current study, we compared measures of ...",Human-RCT-non-drug-intervention,<journal>Journal of consulting and clinical ps...,<title>Treatment engagement and response to CB...
2,20159133,Archives of physical medicine and rehabilitation,Relationship between perceived exertion and ph...,To investigate the strength of the relationshi...,Remaining,<journal>Archives of physical medicine and reh...,<title>Relationship between perceived exertion...
3,11781147,Biochimica et biophysica acta,Characterization of a missense mutation at his...,Genetic defects in pyruvate dehydrogenase comp...,Remaining,<journal>Biochimica et biophysica acta</journa...,<title>Characterization of a missense mutation...
4,31706919,Epilepsy & behavior : E&B,The role of P-glycoprotein (P-gp) and inwardly...,Sudden unexpected death in epilepsy (SUDEP) is...,Non-systematic-review,<journal>Epilepsy & behavior : E&B</journal><t...,<title>The role of P-glycoprotein (P-gp) and i...


### Load key for the OpenAI API 

In [9]:
def load_pass(file_path, key_to_find):
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split("=")
            if len(parts) == 2 and parts[0] == key_to_find:
                found_password = parts[1]
                break
    if found_password:
        print("Found password.")
        return found_password
    else:
        print("Password not found for key:", key_to_find)

Note: You need to create a credentials.txt file with the following content:  
OPENAI=sk-77QXXXXXXXXXXXXXXXXXXXXXXXXXXX  
replace the value after the = sign with your API key.  
Make sure the credentials.txt is added to .gitignore, you don't want to put your password on Git!

In [10]:
openai.api_key = load_pass("./credentials.txt", "OPENAI")


Found password.


In [11]:
client = OpenAI(api_key=openai.api_key)

### Query GPT

To change the task the model is solving, you need to change the text of the prompt and the content text of the system role.  
To change the GPT model used, you need to change the text of the model name when initiating the openai API.  
The function gets as input the input_raw_text, that will be text for information extraction or classification.

In [12]:
import time

DEFAULT_TEMPERATURE = 0
DEFAULT_MAX_TOKENS = 500
DEFAULT_MODEL = "gpt-3.5-turbo"

def query_gpt(input_raw_text, prompt_text, gpt_model="gpt-3.5-turbo", temperature=0, max_retries=5, retry_delay=3):
    # CHANGE gpt_model to the desired model name, see https://platform.openai.com/docs/models (gpt-3.5-turbo and gpt-4-turbo-preview)
    
    # Add a delay at the beginning of the function to avoid overloading the API if there are multiple calls
    # time.sleep(10)  

    system_msg = f"""
    You are an expert assistant specialized in text classification of PubMed abstracts. """

    retries = 0
    while retries < max_retries:
        print("Trying to call OpenAI API...")
        try:
            completion = client.chat.completions.create(
                model=gpt_model,  
                response_format={"type": "json_object"},
                temperature=temperature,
                #max_tokens=2000,
                messages=[
                    {"role": "system", "content": system_msg},
                    {"role": "user", "content": prompt_text + input_raw_text}
                ]
            )
            return completion.choices[0].message.content
        except Exception as e:
            # Handle API error, e.g., retry or log
            print(f"OpenAI API returned an error: {e}")
            time.sleep(retry_delay)  # Wait before retrying
            retries += 1

    raise RuntimeError("Max retries reached. Unable to complete the API call.")


In [13]:
# Define a function to apply GPT queries with a progress bar
def apply_gpt_with_progress(data_series, prompt_text, model="gpt-3.5-turbo"):
    results = []
    total_items = len(data_series)
    # Create a tqdm progress bar
    with tqdm(total=total_items, desc=f"Processing dataset") as pbar:
        for text in data_series:
            result = query_gpt(text, prompt_text, model)
            results.append(result)
            pbar.update(1)  # Update the progress bar

    return results

# Read prompts from file and query GPT

In [14]:
json_file_path = "./prompt_strategies_shirin.json"
# Load the JSON file
with open(json_file_path, 'r') as file:
    prompts_data = json.load(file)

In [15]:
df.head() #show first few lines of df
#print(df) #shows whole df

,pmid,journal_name,title,abstract,accepted_label,input_journal_title_abstract,input_title_abstract
0,18379746,Der Radiologe,[Pediatric stroke].,Stroke in childhood has gained increasingly mo...,Non-systematic-review,<journal>Der Radiologe</journal><title>[Pediat...,<title>[Pediatric stroke].</title><abstract>St...
1,24660674,Journal of consulting and clinical psychology,Treatment engagement and response to CBT among...,"In the current study, we compared measures of ...",Human-RCT-non-drug-intervention,<journal>Journal of consulting and clinical ps...,<title>Treatment engagement and response to CB...
2,20159133,Archives of physical medicine and rehabilitation,Relationship between perceived exertion and ph...,To investigate the strength of the relationshi...,Remaining,<journal>Archives of physical medicine and reh...,<title>Relationship between perceived exertion...
3,11781147,Biochimica et biophysica acta,Characterization of a missense mutation at his...,Genetic defects in pyruvate dehydrogenase comp...,Remaining,<journal>Biochimica et biophysica acta</journa...,<title>Characterization of a missense mutation...
4,31706919,Epilepsy & behavior : E&B,The role of P-glycoprotein (P-gp) and inwardly...,Sudden unexpected death in epilepsy (SUDEP) is...,Non-systematic-review,<journal>Epilepsy & behavior : E&B</journal><t...,<title>The role of P-glycoprotein (P-gp) and i...


## Run different prompts over the data

In [16]:
# Add the IDs of Prompts that you want to test
prompt_ids_to_test = ["P10", "P11"] #, "P2", "P3", "P4"
model = "gpt-3.5-turbo"

for prompt in prompts_data["prompts"]:
    prompt_id = prompt["id"]
    prompt_text = prompt["text"]

    if prompt_id in prompt_ids_to_test:
        # Apply GPT predictions
        df[f'gpt_predictions_{prompt_id}_raw'] = apply_gpt_with_progress(df['input_journal_title_abstract'], prompt_text, model)
        ## the below includes error handling in case the json formatting did not work as expected
        df[f'gpt_predictions_{prompt_id}'] = df[f'gpt_predictions_{prompt_id}_raw'].apply(
            lambda x: json.loads(x)['gpt_label'] if isinstance(x, str) and 'gpt_label' in json.loads(x) else x
        )
        df.to_csv(f"predictions/{model}_outputs_{'_'.join(prompt_ids_to_test)}.csv") # saving after each prompt strategy has been run, in order not to loose information in case a later strategy fails
    else:
        print(f"Skipping prompt {prompt_id}")
        

Skipping prompt P1
Skipping prompt P2
Skipping prompt P2_1
Skipping prompt P3
Skipping prompt P3_1
Skipping prompt P3_2
Skipping prompt P3_3
Skipping prompt P3_4
Skipping prompt P4
Skipping prompt P4_1
Skipping prompt P4_2
Skipping prompt P5
Skipping prompt P6
Skipping prompt P7
Skipping prompt P8
Skipping prompt P9


Processing dataset:   0%|                     | 0/1996 [00:00<?, ?it/s]

Trying to call OpenAI API...


Processing dataset:   0%|             | 1/1996 [00:00<27:35,  1.20it/s]

Trying to call OpenAI API...


...

Trying to call OpenAI API...


Processing dataset: 100%|█████████▉| 1995/1996 [26:27<00:00,  1.32it/s]

Trying to call OpenAI API...


Processing dataset: 100%|██████████| 1996/1996 [26:28<00:00,  1.26it/s]

Skipping prompt P11_1
Skipping prompt P11_2
Skipping prompt P11_3
Skipping prompt P11_4


In [17]:
df.head()

,pmid,journal_name,title,abstract,accepted_label,input_journal_title_abstract,input_title_abstract,gpt_predictions_P10_raw,gpt_predictions_P10,gpt_predictions_P11_raw,gpt_predictions_P11
0,18379746,Der Radiologe,[Pediatric stroke].,Stroke in childhood has gained increasingly mo...,Non-systematic-review,<journal>Der Radiologe</journal><title>[Pediat...,<title>[Pediatric stroke].</title><abstract>St...,"{\n ""gpt_label"": ""Remaining""\n}",Remaining,"{\n ""gpt_label"": ""Remaining""\n}",Remaining
1,24660674,Journal of consulting and clinical psychology,Treatment engagement and response to CBT among...,"In the current study, we compared measures of ...",Human-RCT-non-drug-intervention,<journal>Journal of consulting and clinical ps...,<title>Treatment engagement and response to CB...,"{\n ""gpt_label"": ""Human-RCT-non-drug-interv...",Human-RCT-non-drug-intervention,"{\n ""gpt_label"": ""Human-RCT-drug-interventi...",Human-RCT-drug-intervention
2,20159133,Archives of physical medicine and rehabilitation,Relationship between perceived exertion and ph...,To investigate the strength of the relationshi...,Remaining,<journal>Archives of physical medicine and reh...,<title>Relationship between perceived exertion...,"{\n ""gpt_label"": ""Human-non-RCT-non-drug-in...",Human-non-RCT-non-drug-intervention,"{\n ""gpt_label"": ""Human-non-RCT-non-drug-in...",Human-non-RCT-non-drug-intervention
3,11781147,Biochimica et biophysica acta,Characterization of a missense mutation at his...,Genetic defects in pyruvate dehydrogenase comp...,Remaining,<journal>Biochimica et biophysica acta</journa...,<title>Characterization of a missense mutation...,"{\n ""gpt_label"": ""Human-case-report""\n}",Human-case-report,"{\n ""gpt_label"": ""Human-case-report""\n}",Human-case-report
4,31706919,Epilepsy & behavior : E&B,The role of P-glycoprotein (P-gp) and inwardly...,Sudden unexpected death in epilepsy (SUDEP) is...,Non-systematic-review,<journal>Epilepsy & behavior : E&B</journal><t...,<title>The role of P-glycoprotein (P-gp) and i...,"{\n ""gpt_label"": ""Human-systematic-review""\n}",Human-systematic-review,"{\n ""gpt_label"": ""Animal-drug-intervention""\n}",Animal-drug-intervention


## Evaluate each prompt

In [18]:
labels = ["Human-systematic-review", "Human-RCT-drug-intervention", "Human-RCT-non-drug-intervention", "Human-RCT-non-intervention", "Human-case-report", "Human-non-RCT-drug-intervention", "Human-non-RCT-non-drug-intervention", "Animal-systematic-review", "Animal-drug-intervention", "Animal-non-drug-intervention", "Animal-other", "Non-systematic-review", "In-vitro-study", "Clinical-study-protocol", "Remaining"]

label_to_numerical = {label: i for i, label in enumerate(labels)}
label_to_numerical["label missing"] = -1

In [19]:
def map_label_to_numerical(label):
    # Check if label is a dictionary
    if isinstance(label, dict):
        # Extract the label with the highest score/probability
        highest_label = max(label, key=label.get)
        return label_to_numerical.get(highest_label, -1)
    else:
        # Directly map string labels to numerical IDs
        return label_to_numerical.get(label, -1)
        
# Convert accepted labels to numerical
df['accepted_label_numerical'] = df['accepted_label'].apply(lambda x: label_to_numerical.get(x, -1))

# Initialize a list to hold DataFrame for each report and summary statistics
report_dfs = []
summary_stats = []

# Iterate over each GPT prediction column
for prompt_id in prompt_ids_to_test:
    print("Evaluating ", prompt_id)
    prediction_col = f'gpt_predictions_{prompt_id}'
    
    # Map GPT predictions to numerical values
    df[f'{prediction_col}_numerical'] = df[prediction_col].apply(map_label_to_numerical)
    
    # Extract arrays for evaluation
    y_true = df['accepted_label_numerical'].values
    y_pred = df[f'{prediction_col}_numerical'].values
    
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    accuracy_balanced = balanced_accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred, output_dict=True, zero_division=0, labels=range(len(labels)), target_names=labels)
    
    # Create DataFrame from report
    report_df = pd.DataFrame(report).transpose()
    report_df['Prompt ID'] = prompt_id  # Add column to indicate the prompt ID
    report_dfs.append(report_df)
    
    # Extract summary statistics (average precision, recall, F1)
    summary = report_df.loc['weighted avg', ['precision', 'recall', 'f1-score']].to_dict()
    summary['Prompt ID'] = prompt_id
    summary_stats.append(summary)

# Combine all report DataFrames
all_reports_df = pd.concat(report_dfs)

# Create a summary table for average precision, recall, and F1-score
summary_df = pd.DataFrame(summary_stats)

Evaluating  P10
Evaluating  P11


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2394: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:2394: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [20]:
summary_df

,precision,recall,f1-score,Prompt ID
0,0.512124,0.276692,0.241182,P10
1,0.491253,0.304762,0.282315,P11


In [21]:
all_reports_df.to_csv(f"evaluations/{model}_per_class_{'_'.join(prompt_ids_to_test)}.csv") # saving after each prompt strategy has been run, in order not to loose information in case a later strategy fails
summary_df.to_csv(f"evaluations/{model}_summary_{'_'.join(prompt_ids_to_test)}.csv") # saving after each prompt strategy has been run, in order not to loose information in case a later strategy fails

In [22]:
all_reports_df

,precision,recall,f1-score,support,Prompt ID
Human-systematic-review,0.177049,0.900000,0.295890,60.0,P10
Human-RCT-drug-intervention,0.435897,0.739130,0.548387,23.0,P10
Human-RCT-non-drug-intervention,0.603774,0.842105,0.703297,38.0,P10
Human-RCT-non-intervention,0.000000,0.000000,0.000000,4.0,P10
Human-case-report,0.235537,0.991304,0.380634,115.0,P10
Human-non-RCT-drug-intervention,0.600000,0.209302,0.310345,129.0,P10
Human-non-RCT-non-drug-intervention,0.124764,0.423077,0.192701,156.0,P10
Animal-systematic-review,0.000000,0.000000,0.000000,0.0,P10
Animal-drug-intervention,0.586957,0.835052,0.689362,97.0,P10
Animal-non-drug-intervention,0.230769,0.333333,0.272727,27.0,P10
